In [11]:
using NLsolve
using LinearAlgebra
using QuantumOptics.QuantumOpticsBase
import PyPlot
const plt = PyPlot;

# Setup

In [536]:
# https://groups.google.com/g/julia-users/c/BswpTOoHdfA 
function lpermutations(a)
    b = Vector()
    sort!(a)
    n = length(a)
    while(true)
        push!(b, copy(a))    
        j = n-1
        while(a[j] >= a[j+1])
            j -= 1
            j == 0 && return(b)
        end
        l=n
        while(a[j] >= a[l])
            l -= 1
        end
        tmp = a[l]
        a[l] = a[j]
        a[j] = tmp
        k = j+1
        l = n
        while(k < l)
            tmp = a[k]
            a[k] = a[l]
            a[l] = tmp
            k += 1
            l -= 1
        end
    end
end

# https://stackoverflow.com/questions/37101324/how-to-zero-out-small-values-in-an-array#answer-37101609
function sparsify(x, eps) 
    if abs(x) < eps
        return 0.0
    elseif abs(real(x)) < eps
        return imag(x) * 1im
    elseif abs(imag(x)) < eps
        return real(x)
    end
    return x
end

function sparsify(x) 
    eps = 1e-8
    if abs(x) < eps
        return 0.0
    elseif abs(real(x)) < eps
        return imag(x) * 1im
    elseif abs(imag(x)) < eps
        return real(x)
    end
    return x
end

##################################################################################################################################
# Convenience functions for computing Tₘₙ(θ,ϕ) and Tₘₙ⁻¹(θ,ϕ) as defined in http://dx.doi.org/10.1364/OPTICA.3.001460 for n=m+1
##################################################################################################################################
function Tdiagonal(N, m, θ, ϕ; inv=false, diag=1)
    values = diag * ones(ComplexF64, N)
    values[m] = exp(1im * (-1)^Int(inv) * ϕ) * cos(θ)
    values[m+1] = cos(θ)
    return values
end

function Tlowerdiagonal(N, m, θ, ϕ; inv=false)
    values = zeros(ComplexF64, N-1)
    if inv
        values[m] = -sin(θ)
    else
        values[m] = exp(1im * ϕ) * sin(θ)
    end
    return values
end

function Tupperdiagonal(N, m, θ, ϕ; inv=false)
    values = zeros(ComplexF64, N-1)
    if inv
        values[m] = exp(-1im * ϕ) * sin(θ)
    else   
        values[m] = -sin(θ)
    end
    return values
end

"""
    Tmn(N::Int, m::Int, θ::Real, ϕ::Real; inv::Bool=false)

Compute Tₘₙ(θ,ϕ) of dimension N×N (or Tₘₙ⁻¹(θ,ϕ) when `inv=true`) as described in [this paper](http://dx.doi.org/10.1364/OPTICA.3.001460) when `n=m+1`.
"""
function Tmn(N::Int, m::Int, θ::Real, ϕ::Real; inv::Bool=false, convention="clements")
    if convention == "clements"
        diagm(0 => Tdiagonal(N, m, θ, ϕ, inv=inv), 
             -1 => Tlowerdiagonal(N, m, θ, ϕ, inv=inv), 
              1 => Tupperdiagonal(N, m, θ, ϕ, inv=inv)
        )
    elseif convention == "asymmetric"
         ϕ += π
        diagm(0 => Tdiagonal(N, m, θ, ϕ, inv=inv, diag=-1), 
             -1 => Tlowerdiagonal(N, m, θ, ϕ, inv=inv), 
              1 => Tupperdiagonal(N, m, θ, ϕ, inv=inv)
        )
    end
end

function Tmn(N::Int, m::Int, n::Int, θ::Real, ϕ::Real; inv::Bool=false, convention="clements")
    a = 1
    if convention == "clements"
        nothing
    elseif convention == "asymmetric"
        a = -1
        ϕ += π
    end
    T = diagm(0 => a * Complex[1. for _ in 1:N])
    T[m, m] = exp(1im * (-1)^Int(inv) * ϕ) * cos(θ)
    T[n, n] = cos(θ)
    if inv
        T[m, n] = exp(-1im * ϕ) * sin(θ)
        T[n, m] = -sin(θ)
    else
        T[m, n] = -sin(θ)
        T[n, m] = exp(1im * ϕ) * sin(θ)
    end
    return T
end

"""
    swap(N, m, n)
Returns NxN unitary description of operation that swaps input m with n on outputs.
"""
function swap(N, m, n)
    U = diagm(0 => Complex[i == m || i == n ? 0 : 1 for i in 1:N])
    U[m, n] = 1.
    U[n, m] = 1.
    return U
end

swap

In [150]:
b = FockBasis(4);
I = identityoperator(b)
a = [mapreduce(⊗, [i == j ? create(b) : I for i in 1:8]) for j in 1:8]
ground_state = reduce(⊗, [fockstate(b, 0) for _ in 1:8])

function output_state(input_state)
    # eg. input_state = [(1, [0, 0, 0, 0]), (-1, [1, 1, 1, 1])]
    output_state = ground_state * 0
    zero_array = a[1] * 0
    id = identityoperator(a[1].basis_l)
    for component in input_state
        prefactor, state = component
        total_operator = copy(id)
        for (i, val) in enumerate(state)
            operator = copy(zero_array)
            for (j, el) in enumerate(view(U, :, i))
                operator += el * a[2j - 1 + val]
            end
            total_operator *= operator
        end
        output_state += prefactor * total_operator * ground_state
    end
    return output_state
end          

output_state (generic function with 2 methods)

In [193]:
function W(x...) 
    lp = lpermutations([0, 0, 0, 1])
    return [(x[i] / 2, lp[i]) for i in 1:length(x)]
end

function M(x...) 
    lp = lpermutations([1, 1, 1, 0])
    return [(x[i] / 2, lp[i]) for i in 1:length(x)]
end

W1 = W(1,1,1,1)
W2 = W(1,1,-1,-1)
W3 = W(1,-1,-1,1)
W4 = W(1,-1,1,-1)
M1 = M(1,1,1,1)
M2 = M(1,1,-1,-1)
M3 = M(1,-1,-1,1)
M4 = M(1,-1,1,-1)
GHZ(x) = [(1/√2, [0, 0, 0, 0]), ((-1)^((x%2)+1)/√2, [1, 1, 1, 1])]
K(x) = [(1, lpermutations([0, 0, 1, 1])[x])]

K (generic function with 1 method)

In [590]:
function statetostring(state)
    s = ""
    for (i, el) in enumerate(state)
        el == 0 && continue
        if i%2 == 1
            s *= "H$(i÷2 + 1) "
        else
            s *= "V$(i÷2) "
        end
    end
    return s
end

function clicks(state)
    results = []
    permutations = lpermutations([0, 0, 0, 0, 1, 1, 1, 1])
    for permutation in permutations
        mstate = mapreduce(x -> x == 0 ? fockstate(b, 0) : fockstate(b, 1), ⊗, permutation)
        prob = round(abs(mstate' * state)^2, digits=4)
        prob > 1e-6 && push!(results, [statetostring(permutation), prob, permutation])
    end
    return results
end

function commonelements(clicks1, clicks2)
    for c1 in clicks1, c2 in clicks2
        c1 == c2 && return true
    end
    return false
end

commonelements (generic function with 1 method)

# Calculations

In [541]:
# Symmetric 4-port beam splitters are constrained to the following form
ϕv = 0
U = ComplexF64[
    1         1         1         1
    1   exp(1im * ϕv)  -1  -exp(1im * ϕv)
    1        -1         1        -1
    1  -exp(1im * ϕv)  -1   exp(1im * ϕv)
] / 2
# Uk = kron(U, [1 0; 0 1])
# sparsify.(convert(Array{Int}, 2U))
sparsify.(U)

4×4 Matrix{Float64}:
 0.5   0.5   0.5   0.5
 0.5   0.5  -0.5  -0.5
 0.5  -0.5   0.5  -0.5
 0.5  -0.5  -0.5   0.5

In [591]:
states = [
    W1, W2, W3, W4, 
    M1, M2, M3, M4, 
    GHZ(1), GHZ(2), 
    K(1), K(2), K(3), K(4), K(5), K(6)
];

In [592]:
r = []
for state in states
     push!(r, clicks(output_state(state)))
end

In [601]:
for (i, x) in enumerate(r[1])
    x[1] = join(sort(split(x[1], " ")), " ")
    println(x)
end

Any[" H2 H3 H4 V1", 0.0625, [0, 1, 1, 0, 1, 0, 1, 0]]
Any[" H1 H3 H4 V2", 0.0625, [1, 0, 0, 1, 1, 0, 1, 0]]
Any[" H1 H2 H4 V3", 0.0625, [1, 0, 1, 0, 0, 1, 1, 0]]
Any[" H1 H2 H3 V4", 0.0625, [1, 0, 1, 0, 1, 0, 0, 1]]


In [602]:
for (i, x) in enumerate(r[2])
    x[1] = join(sort(split(x[1], " ")), " ")
    println(x)
end

Any[" H2 H3 H4 V2", 0.0625, [0, 0, 1, 1, 1, 0, 1, 0]]
Any[" H1 H2 H4 V4", 0.0625, [1, 0, 1, 0, 0, 0, 1, 1]]
Any[" H1 H2 H3 V3", 0.0625, [1, 0, 1, 0, 1, 1, 0, 0]]
Any[" H1 H3 H4 V1", 0.0625, [1, 1, 0, 0, 1, 0, 1, 0]]


In [606]:
4/16

0.25

In [603]:
for (i, x) in enumerate(r[3])
    x[1] = join(sort(split(x[1], " ")), " ")
    println(x)
end

Any[" H2 H3 H4 V4", 0.0625, [0, 0, 1, 0, 1, 0, 1, 1]]
Any[" H1 H3 H4 V3", 0.0625, [1, 0, 0, 0, 1, 1, 1, 0]]
Any[" H1 H2 H4 V2", 0.0625, [1, 0, 1, 1, 0, 0, 1, 0]]
Any[" H1 H2 H3 V1", 0.0625, [1, 1, 1, 0, 1, 0, 0, 0]]


In [604]:
for (i, x) in enumerate(r[4])
    x[1] = join(sort(split(x[1], " ")), " ")
    println(x)
end

Any[" H2 H3 H4 V3", 0.0625, [0, 0, 1, 0, 1, 1, 1, 0]]
Any[" H1 H3 H4 V4", 0.0625, [1, 0, 0, 0, 1, 0, 1, 1]]
Any[" H1 H2 H3 V2", 0.0625, [1, 0, 1, 1, 1, 0, 0, 0]]
Any[" H1 H2 H4 V1", 0.0625, [1, 1, 1, 0, 0, 0, 1, 0]]


In [544]:
for i in 1:length(states), j in i+1:length(states)
    if commonelements(map(x->x[3], r[i]), map(x->x[3], r[j])) 
        println("i:$i, j:$j")
    end
end

i:9, j:10
i:11, j:16
i:12, j:15
i:13, j:14


In [545]:
D = diagm(0 => [1, 1, 1, 1])

Ubuilt = sparsify.(
    D *
    Tmn(4, 2, 0, 0) *     # identity
    Tmn(4, 3, π/4, 0, convention="asymmetric") * 
    Tmn(4, 1, π/4, 0, convention="asymmetric") * 
    swap(4, 2, 3) * 
    Tmn(4, 3, π/4, 0, convention="asymmetric") * 
    Tmn(4, 1, π/4, 0, convention="asymmetric") 
    
)
convert(Array{Int}, 2* round.(Ubuilt, digits=2))
# Ubuilt

4×4 Matrix{Int64}:
 1   1   1   1
 1   1  -1  -1
 1  -1   1  -1
 1  -1  -1   1

In [548]:
U * [1 0 0 0]'

4×1 Matrix{ComplexF64}:
 0.5 + 0.0im
 0.5 + 0.0im
 0.5 + 0.0im
 0.5 + 0.0im

In [549]:
U * [0 1 0 0]'

4×1 Matrix{ComplexF64}:
  0.5 + 0.0im
  0.5 + 0.0im
 -0.5 + 0.0im
 -0.5 + 0.0im